# Домашняя работа 

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов 

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ds_salaries.csv")

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
y = df["salary_in_usd"]
X = df.drop(["salary_in_usd", "salary"], axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [8]:
cat_features_mask = (X_train.dtypes == "object").values
X_train_no_cat = X_train[X_train.columns[~cat_features_mask]]
X_test_no_cat = X_test[X_test.columns[~cat_features_mask]]

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

lr = LinearRegression().fit(X_train_no_cat, y_train)
print(f"RMSE for Train: {mean_squared_error(y_train, lr.predict(X_train_no_cat), squared=False)}")
print(f"RMSE for Test: {mean_squared_error(y_test, lr.predict(X_test_no_cat), squared=False)}")
print(f"MAPE for Train: {mean_absolute_percentage_error(y_train, lr.predict(X_train_no_cat))}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, lr.predict(X_test_no_cat))}")

RMSE for Train: 63492.37249985216
RMSE for Test: 61402.466607999
MAPE for Train: 0.6557137646822458
MAPE for Test: 0.7056558772008524


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [10]:
from catboost import Pool, CatBoostRegressor

In [11]:
pool = Pool(data=X_train, label=y_train, cat_features=list(X_train.columns[cat_features_mask]))

In [12]:
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, random_state=42)
model.fit(pool, eval_set=(X_val, y_val), verbose=False)
print(f"RMSE for Train: {mean_squared_error(y_train, model.predict(X_train), squared=False)}")
print(f"RMSE for Test: {mean_squared_error(y_test, model.predict(X_test), squared=False)}")
print(f"MAPE for Train: {mean_absolute_percentage_error(y_train, model.predict(X_train))}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, model.predict(X_test))}")

RMSE for Train: 48393.8732317884
RMSE for Test: 49194.547426365374
MAPE for Train: 0.38139265080525814
MAPE for Test: 0.45041816986721084


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

Выведем все колонки категориальных признаков:

In [13]:
X_train[X_train.columns[cat_features_mask]].head()

,experience_level,employment_type,job_title,salary_currency,employee_residence,company_location,company_size
3379,SE,FT,Data Scientist,USD,US,US,M
2356,SE,FT,Applied Scientist,USD,US,US,M
3275,EX,FT,Data Analyst,USD,US,US,M
3044,SE,FT,Data Engineer,USD,US,US,M
2018,SE,FT,BI Developer,USD,US,US,M


Преобразуем с помощью **OneHotEncoding** признаки, у которых небольшой диапазон значений:

In [14]:
X_cat = pd.get_dummies(X[['experience_level', 'employment_type', 'company_size']], drop_first=True, sparse=True)
X_cat.head()

,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,company_size_M,company_size_S
0,0,0,1,0,1,0,0,0
1,0,1,0,0,0,0,0,1
2,0,1,0,0,0,0,0,1
3,0,0,1,0,1,0,1,0
4,0,0,1,0,1,0,1,0


Преобразуем с помощью **LabelEncoder:**

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()
categ = ['job_title', 'salary_currency']
X_cat[categ] = X[categ].apply(le.fit_transform)
X_cat.head()

,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,company_size_M,company_size_S,job_title,salary_currency
0,0,0,1,0,1,0,0,0,84,7
1,0,1,0,0,0,0,0,1,66,19
2,0,1,0,0,0,0,0,1,66,19
3,0,0,1,0,1,0,1,0,47,19
4,0,0,1,0,1,0,1,0,47,19


Преобразуем с помощью **TargetEncoder:**

In [17]:
from category_encoders import TargetEncoder

target = TargetEncoder(cols=['employee_residence', 'company_location'])
X_targ = target.fit_transform(X[['employee_residence', 'company_location']], y)
X_targ.head()

,employee_residence,company_location
0,60654.006652,57942.497114
1,152822.011651,151822.009539
2,152822.011651,151822.009539
3,132230.933430,131924.639077
4,132230.933430,131924.639077


In [18]:
X_update = pd.concat([X[X.columns[~cat_features_mask]], X_cat, X_targ], axis=1)
X_update.head()

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,company_size_M,company_size_S,job_title,salary_currency,employee_residence,company_location
0,2023,100,0,0,1,0,1,0,0,0,84,7,60654.006652,57942.497114
1,2023,100,0,1,0,0,0,0,0,1,66,19,152822.011651,151822.009539
2,2023,100,0,1,0,0,0,0,0,1,66,19,152822.011651,151822.009539
3,2023,100,0,0,1,0,1,0,1,0,47,19,132230.933430,131924.639077
4,2023,100,0,0,1,0,1,0,1,0,47,19,132230.933430,131924.639077


Разделим наши данные на train, val и test:

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_update, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

Обучим линейную модель:

In [20]:
lr = LinearRegression().fit(X_train, y_train)
print(f"RMSE for Train: {mean_squared_error(y_train, lr.predict(X_train), squared=False)}")
print(f"RMSE for Test: {mean_squared_error(y_test, lr.predict(X_test), squared=False)}")
print(f"MAPE for Train: {mean_absolute_percentage_error(y_train, lr.predict(X_train))}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, lr.predict(X_test))}")

RMSE for Train: 51657.439602650324
RMSE for Test: 49785.94815055166
MAPE for Train: 0.3956646385770859
MAPE for Test: 0.43998812144208876


Обучим бустинг:

In [21]:
pool = Pool(data=X_train, label=y_train)

model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, random_seed=42)
model.fit(pool, eval_set=(X_val, y_val), verbose=False)
print(f"RMSE for Train: {mean_squared_error(y_train, model.predict(X_train), squared=False)}")
print(f"RMSE for Test: {mean_squared_error(y_test, model.predict(X_test), squared=False)}")
print(f"MAPE for Train: {mean_absolute_percentage_error(y_train, model.predict(X_train))}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, model.predict(X_test))}")

RMSE for Train: 47769.408774042335
RMSE for Test: 48722.69034995276
MAPE for Train: 0.34926799683334253
MAPE for Test: 0.4046903215211778


Видим, что модель бустинга работает лучше, чем линейная модель.

Для линейной модели обработка категориальных признаков дала весомые улучшения, в то время как ошибка для бустинга изменилась не так сильно.

Далее будем работать с моделью бустинга **catboost**.

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

Добавим в исходный датафрейм столбец с  рассчитанной ошибкой RMSE для каждого из объектов

In [22]:
import numpy as np
preds = pd.Series(model.predict(X_test))
rmse = pd.DataFrame(np.sqrt((np.array(y_test) - np.array(preds))**2), columns=['RMSE'])
res = pd.concat([df.reset_index(drop=True), rmse], axis=1)
res.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,RMSE
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L,3117.731121
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S,51199.665356
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S,20750.795425
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M,59629.424640
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M,43297.822470


Выведем топ 20 примеров с наибольшей ошибкой на тесте

In [23]:
res.sort_values(by=['RMSE'], inplace=True, ascending=False)
res.head(20)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,RMSE
228,2023,EX,FT,Head of Data,329500,USD,329500,US,0,US,M,251187.418880
513,2023,SE,FT,Research Scientist,200000,USD,200000,US,0,US,M,211402.169801
52,2023,SE,FT,Data Scientist,218500,USD,218500,US,100,US,M,202535.304764
184,2023,MI,FT,ETL Engineer,70000,GBP,85066,GB,100,GB,M,199364.314712
277,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L,146353.815244
193,2023,SE,FT,Data Scientist,186000,USD,186000,US,0,US,M,142729.821488
586,2023,MI,FT,Data Scientist,60000,GBP,72914,GB,0,GB,M,141990.083993
286,2023,SE,FT,Data Engineer,219535,USD,219535,US,100,US,M,139977.890046
341,2023,EN,FT,Data Engineer,65000,USD,65000,US,0,US,M,138959.693491
169,2023,SE,FT,Analytics Engineer,150000,USD,150000,US,100,US,M,136067.102812


Видим, что для большой ошибки в явными признаками явлвяются 2023 год работы - `work_year = 2023` и full-time рабочий день - `employment_type = FT`, также в большинстве своем это свойственно компаниям среднего размера `company_size = M`.

Для того чтобы улучшить нашу модель, постараемся сгенерировать новые признаки на основе данных знаний.

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

Добавим новый признак - `work_year * company_size` для этого закодируем `company_size` с помощью `LabelEncoder`,а остальные категориальные признаки так же, как и в 4 пункте. После отмасштабируем признаки.

In [49]:
X.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,EUR,ES,100,ES,L
1,2023,MI,CT,ML Engineer,USD,US,100,US,S
2,2023,MI,CT,ML Engineer,USD,US,100,US,S
3,2023,SE,FT,Data Scientist,USD,CA,100,CA,M
4,2023,SE,FT,Data Scientist,USD,CA,100,CA,M


Закодируем категориальные признаки:

In [50]:
X_cat = pd.get_dummies(X[['experience_level', 'employment_type']], drop_first=True, sparse=True)
X_cat.head()

,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT
0,0,0,1,0,1,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,1,0
4,0,0,1,0,1,0


In [51]:
le = LabelEncoder()
categ = ['job_title', 'salary_currency', 'company_size']
X_cat[categ] = X[categ].apply(le.fit_transform)
X_cat.head()

,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title,salary_currency,company_size
0,0,0,1,0,1,0,84,7,0
1,0,1,0,0,0,0,66,19,2
2,0,1,0,0,0,0,66,19,2
3,0,0,1,0,1,0,47,19,1
4,0,0,1,0,1,0,47,19,1


In [52]:
target = TargetEncoder(cols=['employee_residence', 'company_location'])
X_targ = target.fit_transform(X[['employee_residence', 'company_location']], y)
X_targ.head()

,employee_residence,company_location
0,60654.006652,57942.497114
1,152822.011651,151822.009539
2,152822.011651,151822.009539
3,132230.933430,131924.639077
4,132230.933430,131924.639077


In [53]:
X_update = pd.concat([X[X.columns[~cat_features_mask]], X_cat, X_targ], axis=1)
X_update.head()

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title,salary_currency,company_size,employee_residence,company_location
0,2023,100,0,0,1,0,1,0,84,7,0,60654.006652,57942.497114
1,2023,100,0,1,0,0,0,0,66,19,2,152822.011651,151822.009539
2,2023,100,0,1,0,0,0,0,66,19,2,152822.011651,151822.009539
3,2023,100,0,0,1,0,1,0,47,19,1,132230.933430,131924.639077
4,2023,100,0,0,1,0,1,0,47,19,1,132230.933430,131924.639077


Создадим наш новый признак:

In [55]:
X_update["work_year&company_size"] = X_update["work_year"] * X_update["company_size"]
X_update.head()

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title,salary_currency,company_size,employee_residence,company_location,work_year&company_size
0,2023,100,0,0,1,0,1,0,84,7,0,60654.006652,57942.497114,0
1,2023,100,0,1,0,0,0,0,66,19,2,152822.011651,151822.009539,4046
2,2023,100,0,1,0,0,0,0,66,19,2,152822.011651,151822.009539,4046
3,2023,100,0,0,1,0,1,0,47,19,1,132230.933430,131924.639077,2023
4,2023,100,0,0,1,0,1,0,47,19,1,132230.933430,131924.639077,2023


In [59]:
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_update, y, test_size=0.2, random_state=42)
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, test_size=0.5, random_state=42)

Отмасштабируем признаки:

In [63]:
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()
X_exp_train_norm = normalizer.fit_transform(X_exp_train)
X_exp_val_norm = normalizer.transform(X_exp_val)
X_exp_test_norm = normalizer.transform(X_exp_test)

Обучим бустинг:

In [64]:
pool = Pool(data=X_exp_train_norm, label=y_exp_train)

model_exp = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, random_seed=42)
model_exp.fit(pool, eval_set=(X_exp_val_norm, y_exp_val), verbose=False)
print(f"RMSE for Train: {mean_squared_error(y_exp_train, model_exp.predict(X_exp_train_norm), squared=False)}")
print(f"RMSE for Test: {mean_squared_error(y_exp_test, model_exp.predict(X_exp_test_norm), squared=False)}")
print(f"MAPE for Train: {mean_absolute_percentage_error(y_exp_train, model_exp.predict(X_exp_train_norm))}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_exp_test, model_exp.predict(X_exp_test_norm))}")

RMSE for Train: 47693.7971515933
RMSE for Test: 48790.32533948423
MAPE for Train: 0.35143132635327157
MAPE for Test: 0.4133821029824139


**Вывод:**

Видим, что при добавлении нового признака ошибка чуть увеличилась и на MAPE, и на RMSE. Поэтому можем сделать вывод, что наша модель не улучшилась и данный признак пользы не несёт.

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [24]:
y_catboost = df["salary_in_usd"]
X_catboost = df.drop(["salary_in_usd", "salary"], axis=1)

X_catboost_train, X_catboost_test, y_catboost_train, y_catboost_test = train_test_split(X_catboost, y_catboost, test_size=0.2, random_state=42)
X_catboost_train, X_catboost_val, y_catboost_train, y_catboost_val = train_test_split(X_catboost_train, y_catboost_train, test_size=0.5, random_state=42)

In [25]:
cat_features_mask = (X_catboost_train.dtypes == "object").values

pool_catboost = Pool(data=X_catboost_train, label=y_catboost_train,
                     cat_features=list(X_catboost_train.columns[cat_features_mask]))

In [26]:
catboost = CatBoostRegressor(cat_features=list(X_catboost_train.columns[cat_features_mask]))

grid = {
    "learning_rate": [0.03, 0.05, 0.1],
    "depth": range(3, 9),
    "iterations": [100, 150, 200]
}


grid_search_result = catboost.grid_search(
    grid, X=X_catboost_train, y=y_catboost_train, verbose=False)

0:	learn: 148473.6002133	test: 150410.6180275	best: 150410.6180275 (0)	total: 18.9ms	remaining: 1.87s
1:	learn: 144885.1919236	test: 146823.2547773	best: 146823.2547773 (1)	total: 22.9ms	remaining: 1.12s
2:	learn: 141380.0786599	test: 143333.3618308	best: 143333.3618308 (2)	total: 40.9ms	remaining: 1.32s
3:	learn: 137937.0781525	test: 139866.4674581	best: 139866.4674581 (3)	total: 58.9ms	remaining: 1.41s
4:	learn: 134613.5468418	test: 136526.8793640	best: 136526.8793640 (4)	total: 84.3ms	remaining: 1.6s
5:	learn: 131489.7653327	test: 133411.7240306	best: 133411.7240306 (5)	total: 112ms	remaining: 1.76s
6:	learn: 128457.3982470	test: 130372.6565991	best: 130372.6565991 (6)	total: 137ms	remaining: 1.82s
7:	learn: 125570.0274844	test: 127484.2822042	best: 127484.2822042 (7)	total: 161ms	remaining: 1.85s
8:	learn: 122777.8711780	test: 124756.0852480	best: 124756.0852480 (8)	total: 188ms	remaining: 1.9s
9:	learn: 120105.6610159	test: 122085.5094490	best: 122085.5094490 (9)	total: 197ms	rema

84:	learn: 54419.3623633	test: 57280.8801327	best: 57280.8801327 (84)	total: 1.95s	remaining: 344ms
85:	learn: 54331.5768667	test: 57182.8539569	best: 57182.8539569 (85)	total: 1.97s	remaining: 321ms
86:	learn: 54138.6819763	test: 57031.9154289	best: 57031.9154289 (86)	total: 2s	remaining: 299ms
87:	learn: 53943.6294866	test: 56846.3271787	best: 56846.3271787 (87)	total: 2.02s	remaining: 276ms
88:	learn: 53770.2688497	test: 56678.9246955	best: 56678.9246955 (88)	total: 2.05s	remaining: 253ms
89:	learn: 53597.5972787	test: 56504.9941473	best: 56504.9941473 (89)	total: 2.07s	remaining: 230ms
90:	learn: 53433.6394012	test: 56350.7654239	best: 56350.7654239 (90)	total: 2.1s	remaining: 207ms
91:	learn: 53282.8871655	test: 56214.1137200	best: 56214.1137200 (91)	total: 2.12s	remaining: 184ms
92:	learn: 53149.6264882	test: 56108.8545437	best: 56108.8545437 (92)	total: 2.14s	remaining: 161ms
93:	learn: 53039.2476901	test: 56014.3624825	best: 56014.3624825 (93)	total: 2.17s	remaining: 139ms
94:	

74:	learn: 50889.2429931	test: 54598.4262860	best: 54598.4262860 (74)	total: 2.03s	remaining: 676ms
75:	learn: 50748.7622398	test: 54468.4145431	best: 54468.4145431 (75)	total: 2.05s	remaining: 648ms
76:	learn: 50693.1387737	test: 54419.2652035	best: 54419.2652035 (76)	total: 2.08s	remaining: 621ms
77:	learn: 50595.8533691	test: 54361.6991585	best: 54361.6991585 (77)	total: 2.1s	remaining: 593ms
78:	learn: 50523.3399454	test: 54303.5745762	best: 54303.5745762 (78)	total: 2.13s	remaining: 566ms
79:	learn: 50407.4691108	test: 54185.0323204	best: 54185.0323204 (79)	total: 2.15s	remaining: 538ms
80:	learn: 50357.8340308	test: 54150.7316592	best: 54150.7316592 (80)	total: 2.18s	remaining: 511ms
81:	learn: 50319.5255726	test: 54131.9994415	best: 54131.9994415 (81)	total: 2.2s	remaining: 484ms
82:	learn: 50292.2775882	test: 54128.8875226	best: 54128.8875226 (82)	total: 2.23s	remaining: 456ms
83:	learn: 50221.9045420	test: 54129.0567688	best: 54128.8875226 (82)	total: 2.25s	remaining: 429ms
84

57:	learn: 49074.2054511	test: 53213.2838410	best: 53213.2838410 (57)	total: 1.5s	remaining: 1.09s
58:	learn: 49042.5768198	test: 53178.6359105	best: 53178.6359105 (58)	total: 1.53s	remaining: 1.06s
59:	learn: 49019.3264449	test: 53214.6250374	best: 53178.6359105 (58)	total: 1.55s	remaining: 1.03s
60:	learn: 49018.4174292	test: 53210.9793513	best: 53178.6359105 (58)	total: 1.57s	remaining: 1.01s
61:	learn: 49007.2778004	test: 53212.4419836	best: 53178.6359105 (58)	total: 1.6s	remaining: 980ms
62:	learn: 48959.3066260	test: 53184.8860307	best: 53178.6359105 (58)	total: 1.62s	remaining: 954ms
63:	learn: 48891.8543941	test: 53159.2178529	best: 53159.2178529 (63)	total: 1.65s	remaining: 929ms
64:	learn: 48891.5281490	test: 53159.7629388	best: 53159.2178529 (63)	total: 1.68s	remaining: 902ms
65:	learn: 48861.7575755	test: 53115.9323319	best: 53115.9323319 (65)	total: 1.7s	remaining: 877ms
66:	learn: 48820.8093982	test: 53078.6054644	best: 53078.6054644 (66)	total: 1.73s	remaining: 852ms
67:

48:	learn: 64299.2326668	test: 67053.3217401	best: 67053.3217401 (48)	total: 1.1s	remaining: 2.26s
49:	learn: 63727.1483981	test: 66475.9062769	best: 66475.9062769 (49)	total: 1.12s	remaining: 2.25s
50:	learn: 63191.6681723	test: 65935.1187757	best: 65935.1187757 (50)	total: 1.15s	remaining: 2.23s
51:	learn: 62663.4547793	test: 65394.8005659	best: 65394.8005659 (51)	total: 1.17s	remaining: 2.21s
52:	learn: 62150.1850884	test: 64903.1089053	best: 64903.1089053 (52)	total: 1.2s	remaining: 2.19s
53:	learn: 61685.4331889	test: 64432.6767021	best: 64432.6767021 (53)	total: 1.22s	remaining: 2.17s
54:	learn: 61243.6962409	test: 63980.3160331	best: 63980.3160331 (54)	total: 1.24s	remaining: 2.13s
55:	learn: 60835.8610149	test: 63560.3263934	best: 63560.3263934 (55)	total: 1.25s	remaining: 2.1s
56:	learn: 60375.5038195	test: 63118.6435726	best: 63118.6435726 (56)	total: 1.28s	remaining: 2.08s
57:	learn: 59973.1504039	test: 62726.3891952	best: 62726.3891952 (57)	total: 1.3s	remaining: 2.06s
58:	

...

198:	learn: 41632.4387438	test: 51140.4960830	best: 51134.6935037 (197)	total: 11.1s	remaining: 55.8ms
199:	learn: 41607.4026110	test: 51154.7877751	best: 51134.6935037 (197)	total: 11.2s	remaining: 0us

bestTest = 51134.6935
bestIteration = 197

Training on fold [1/3]
0:	learn: 140555.0589005	test: 138080.8540556	best: 138080.8540556 (0)	total: 33ms	remaining: 6.57s
1:	learn: 129621.1211599	test: 127083.9528818	best: 127083.9528818 (1)	total: 42.7ms	remaining: 4.23s
2:	learn: 120017.4264715	test: 117379.3639236	best: 117379.3639236 (2)	total: 62.2ms	remaining: 4.08s
3:	learn: 110911.5528342	test: 107987.6844232	best: 107987.6844232 (3)	total: 91.8ms	remaining: 4.5s
4:	learn: 103076.3868794	test: 100299.2853775	best: 100299.2853775 (4)	total: 152ms	remaining: 5.92s
5:	learn: 96066.5442138	test: 93342.7572556	best: 93342.7572556 (5)	total: 190ms	remaining: 6.14s
6:	learn: 89814.7697512	test: 87076.4904679	best: 87076.4904679 (6)	total: 232ms	remaining: 6.39s
7:	learn: 84526.0729055	test

81:	learn: 46178.3978080	test: 49633.7965995	best: 49615.3863873 (73)	total: 4.3s	remaining: 6.18s
82:	learn: 46142.2205045	test: 49643.0700527	best: 49615.3863873 (73)	total: 4.36s	remaining: 6.14s
83:	learn: 46128.0356426	test: 49646.8278749	best: 49615.3863873 (73)	total: 4.42s	remaining: 6.1s
84:	learn: 46096.7098919	test: 49662.7526039	best: 49615.3863873 (73)	total: 4.47s	remaining: 6.05s
85:	learn: 46029.7109625	test: 49654.1605981	best: 49615.3863873 (73)	total: 4.54s	remaining: 6.01s
86:	learn: 45989.8184823	test: 49672.5105207	best: 49615.3863873 (73)	total: 4.59s	remaining: 5.97s
87:	learn: 45963.1271587	test: 49672.4011248	best: 49615.3863873 (73)	total: 4.65s	remaining: 5.92s
88:	learn: 45931.0329593	test: 49670.6882446	best: 49615.3863873 (73)	total: 4.71s	remaining: 5.88s
89:	learn: 45914.2543227	test: 49677.7990792	best: 49615.3863873 (73)	total: 4.77s	remaining: 5.84s
90:	learn: 45818.3583495	test: 49648.8759399	best: 49615.3863873 (73)	total: 4.84s	remaining: 5.79s
91

165:	learn: 43269.4309705	test: 50159.4092577	best: 49615.3863873 (73)	total: 9.28s	remaining: 1.9s
166:	learn: 43253.3045055	test: 50157.5423712	best: 49615.3863873 (73)	total: 9.33s	remaining: 1.84s
167:	learn: 43212.5026465	test: 50176.4462480	best: 49615.3863873 (73)	total: 9.4s	remaining: 1.79s
168:	learn: 43194.2302946	test: 50165.4426753	best: 49615.3863873 (73)	total: 9.46s	remaining: 1.73s
169:	learn: 43098.4679184	test: 50153.1416636	best: 49615.3863873 (73)	total: 9.52s	remaining: 1.68s
170:	learn: 43088.7150218	test: 50173.1974041	best: 49615.3863873 (73)	total: 9.57s	remaining: 1.62s
171:	learn: 43058.5431711	test: 50175.9432405	best: 49615.3863873 (73)	total: 9.63s	remaining: 1.57s
172:	learn: 43044.5853168	test: 50178.0070944	best: 49615.3863873 (73)	total: 9.69s	remaining: 1.51s
173:	learn: 43042.2619476	test: 50181.5960098	best: 49615.3863873 (73)	total: 9.75s	remaining: 1.46s
174:	learn: 43003.7912418	test: 50197.4518881	best: 49615.3863873 (73)	total: 9.8s	remaining:

47:	learn: 47993.3333676	test: 51323.7975669	best: 51323.7975669 (47)	total: 2.36s	remaining: 7.47s
48:	learn: 47868.9354582	test: 51298.3083871	best: 51298.3083871 (48)	total: 2.42s	remaining: 7.46s
49:	learn: 47777.3782124	test: 51257.9186714	best: 51257.9186714 (49)	total: 2.48s	remaining: 7.44s
50:	learn: 47655.5803506	test: 51216.0940725	best: 51216.0940725 (50)	total: 2.54s	remaining: 7.42s
51:	learn: 47546.9631397	test: 51134.7451009	best: 51134.7451009 (51)	total: 2.6s	remaining: 7.41s
52:	learn: 47473.1665101	test: 51120.4877927	best: 51120.4877927 (52)	total: 2.67s	remaining: 7.4s
53:	learn: 47411.2902957	test: 51071.2294914	best: 51071.2294914 (53)	total: 2.73s	remaining: 7.38s
54:	learn: 47288.8210164	test: 50988.0738882	best: 50988.0738882 (54)	total: 2.79s	remaining: 7.35s
55:	learn: 47158.6742694	test: 51000.0989799	best: 50988.0738882 (54)	total: 2.85s	remaining: 7.33s
56:	learn: 47082.5097774	test: 51011.8052234	best: 50988.0738882 (54)	total: 2.91s	remaining: 7.29s
57

129:	learn: 43216.3396836	test: 50930.8492656	best: 50601.3697983 (82)	total: 7.2s	remaining: 3.88s
130:	learn: 43146.9675410	test: 50957.3688042	best: 50601.3697983 (82)	total: 7.27s	remaining: 3.83s
131:	learn: 43068.1158798	test: 50966.8814977	best: 50601.3697983 (82)	total: 7.33s	remaining: 3.77s
132:	learn: 43017.1355438	test: 50960.5968602	best: 50601.3697983 (82)	total: 7.39s	remaining: 3.72s
133:	learn: 43004.5783048	test: 50967.9401202	best: 50601.3697983 (82)	total: 7.45s	remaining: 3.67s
134:	learn: 43001.0502645	test: 50968.6911102	best: 50601.3697983 (82)	total: 7.52s	remaining: 3.62s
135:	learn: 42975.0157587	test: 50960.9537911	best: 50601.3697983 (82)	total: 7.59s	remaining: 3.57s
136:	learn: 42953.6092288	test: 50940.5082910	best: 50601.3697983 (82)	total: 7.65s	remaining: 3.52s
137:	learn: 42898.7924675	test: 50959.1312161	best: 50601.3697983 (82)	total: 7.72s	remaining: 3.47s
138:	learn: 42843.3150760	test: 50989.5460623	best: 50601.3697983 (82)	total: 7.77s	remainin

In [28]:
grid_search_result["params"]

{'depth': 6, 'iterations': 200, 'learning_rate': 0.1}

In [29]:
catboost_best = CatBoostRegressor(**(grid_search_result['params']),
                                  cat_features=list(X_catboost_train.columns[cat_features_mask]))

In [30]:
%%time
catboost_best.fit(pool_catboost, eval_set=(X_catboost_val, y_catboost_val), verbose=False)

CPU times: total: 34.3 s
Wall time: 11.4 s


In [31]:
%%time
catboost_pred = catboost_best.predict(X_catboost_test)

CPU times: total: 62.5 ms
Wall time: 14.2 ms


In [32]:
print(f"RMSE for Test: {mean_squared_error(y_catboost_test, catboost_pred, squared=False)}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_catboost_test, catboost_pred)}")

RMSE for Test: 48801.33996235203
MAPE for Test: 0.4437693028559762


**Вывод:**

С помощью `grid_search` мы нашли оптимальные параметры для нашей модели. Качество итоговой модели чуть хуже, чем качество модели в пункте 4, где мы предобрабатывали категориальные признаки. Время на обучение модели составило 11 секунд, а на предсказание - 14 миллисекунд.

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

Будем использовать train и test из 4 пункта, где мы уже обработали катеогриальные признаки.

In [36]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [37]:
xgboost = XGBRegressor()

params = {
    'learning_rate': [0.03, 0.05, 0.1],
    'max_depth': range(3, 9),
    'n_estimators': [100, 200, 400],
    'min_child_weight': range(1, 6, 2),
    'gamma': [0.5, 0.7, 0.9]
}

grid = GridSearchCV(xgboost, params, cv=3)

grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'gamma': [0.5, 0.7, 0.9],
                         'learning_rate': [0.03, 0.05, 0.1],
                         'max_depth': range(3, 9),
                         'min_child_weight': range(1, 6, 2),
                         'n_estimators': [100, 200, 400]})

In [38]:
grid.best_params_, grid.best_score_

({'gamma': 0.5,
  'learning_rate': 0.03,
  'max_depth': 4,
  'min_child_weight': 1,
  'n_estimators': 200},
 0.426068659668197)

In [39]:
xgb_best = XGBRegressor(**(grid.best_params_))

In [40]:
%%time
xgb_best.fit(X_train, y_train)

CPU times: total: 1.98 s
Wall time: 352 ms


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [41]:
%%time
xgb_pred = xgb_best.predict(X_test)

CPU times: total: 78.1 ms
Wall time: 15.8 ms


In [42]:
print(f"RMSE for Test: {mean_squared_error(y_test, xgb_pred, squared=False)}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, xgb_pred)}")

RMSE for Test: 48658.18595259344
MAPE for Test: 0.39245972856577244


**Вывод:**

С помощью `GridSearchCV` мы нашли оптимальные параметры для нашей модели. Время на обучение модели составило 352 миллисекунд, а на предсказание - 15 миллисекунд. Качество модели является одной из лучших среди всех моделей, составленных ранее, так как ошибки RMSE и MAPE достаточно малы.

## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

Будем использовать train и test из 4 пункта, где мы уже обработали катеогриальные признаки.

In [43]:
from lightgbm import LGBMRegressor

estimator = LGBMRegressor()

param_grid = {
    "learning_rate": [0.03, 0.05, 0.1],
    'max_depth': range(3, 9),
    "n_estimators": [100, 150, 200],
    "num_leaves": [15, 20, 30]
    }

grid = GridSearchCV(estimator, param_grid, cv=3)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.03, 0.05, 0.1],
                         'max_depth': range(3, 9),
                         'n_estimators': [100, 150, 200],
                         'num_leaves': [15, 20, 30]})

In [44]:
grid.best_params_, grid.best_score_

({'learning_rate': 0.03,
  'max_depth': 6,
  'n_estimators': 100,
  'num_leaves': 30},
 0.41543702017921175)

In [45]:
gbm_best = LGBMRegressor(**(grid.best_params_))

In [46]:
%%time
gbm_best.fit(X_train, y_train, eval_set=(X_val, y_val))

[1]	valid_0's l2: 3.63269e+09
[2]	valid_0's l2: 3.54848e+09
[3]	valid_0's l2: 3.46909e+09
[4]	valid_0's l2: 3.39469e+09
[5]	valid_0's l2: 3.32423e+09
[6]	valid_0's l2: 3.25794e+09
[7]	valid_0's l2: 3.19563e+09
[8]	valid_0's l2: 3.13714e+09
[9]	valid_0's l2: 3.08159e+09
[10]	valid_0's l2: 3.02933e+09
[11]	valid_0's l2: 2.9804e+09
[12]	valid_0's l2: 2.93416e+09
[13]	valid_0's l2: 2.89091e+09
[14]	valid_0's l2: 2.85006e+09
[15]	valid_0's l2: 2.81183e+09
[16]	valid_0's l2: 2.77565e+09
[17]	valid_0's l2: 2.74004e+09
[18]	valid_0's l2: 2.70664e+09
[19]	valid_0's l2: 2.67455e+09
[20]	valid_0's l2: 2.64427e+09
[21]	valid_0's l2: 2.61627e+09
[22]	valid_0's l2: 2.58967e+09
[23]	valid_0's l2: 2.56501e+09
[24]	valid_0's l2: 2.54216e+09
[25]	valid_0's l2: 2.51959e+09
[26]	valid_0's l2: 2.49888e+09
[27]	valid_0's l2: 2.47885e+09
[28]	valid_0's l2: 2.46065e+09
[29]	valid_0's l2: 2.44291e+09
[30]	valid_0's l2: 2.42626e+09
[31]	valid_0's l2: 2.41055e+09
[32]	valid_0's l2: 2.39574e+09
[33]	valid_0's l2:

LGBMRegressor(learning_rate=0.03, max_depth=6, num_leaves=30)

In [47]:
%%time
gbm_pred = gbm_best.predict(X_test)

CPU times: total: 93.8 ms
Wall time: 9.01 ms


In [48]:
print(f"RMSE for Test: {mean_squared_error(y_test, gbm_pred, squared=False)}")
print(f"MAPE for Test: {mean_absolute_percentage_error(y_test, gbm_pred)}")

RMSE for Test: 47743.81410627024
MAPE for Test: 0.3836426436783541


**Вывод:**

С помощью `GridSearchCV` мы нашли оптимальные параметры для нашей модели. Время на обучение модели составило 169 миллисекунд, а на предсказание - 9 миллисекунд. Качество модели является одной из лучших среди всех моделей, составленных ранее, так как ошибки RMSE и MAPE достаточно малы.

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

Среди моделей xgboost, lightgbm и catboost наиболее оптимальной оказалась **lightgbm**. У неё наименьшая ошибка среди трёх перечисленных модлей бустинга.